# Import library


In [1]:
import torch
import math
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import random
import torchvision

from sklearn.model_selection import train_test_split
from typing import Any
from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.tensorboard import SummaryWriter
from PIL import Image
from sklearn.metrics import accuracy_score
from tqdm import tqdm

# Setup device and tensorboard


In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.manual_seed(42)

# Customized Dataset


In [3]:
class ImageDataset(Dataset):
    def __init__(self, list_root_dir: list, transform=None) -> None:
        super().__init__()
        self.list_root_dir = list_root_dir
        self.transform = transform

        self.image_paths = []
        for root_dir in list_root_dir:
            self.image_paths.extend(
                glob.glob(os.path.join(root_dir, "with", "*.jpg")))
            self.image_paths.extend(
                glob.glob(os.path.join(root_dir, "without", "*.jpg")))
        random.shuffle(self.image_paths)

    def __len__(self) -> int:
        return len(self.image_paths)

    def __getitem__(self, index) -> Any:
        image_path = self.image_paths[index]

        label = 1 if image_path.split(os.sep)[-2] == "with" else 0
        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)

        return [image, label]

# Train one epoch


In [4]:
def train_model(model, loss_function, optimizer, train_loader, writer, epoch=1):
    model.train()
    running_loss = 0.0
    total_samples = 0
    y_true = []
    y_pred = []
    for data in train_loader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        # Clear gradient
        optimizer.zero_grad()
        # Calculate logits
        outputs = model(images)
        # Calculate loss
        loss = loss_function(outputs, labels)
        # Calculate gradient from loss
        loss.backward()
        # Update weight
        optimizer.step()

        # Calculate loss
        running_loss += loss.item() * images.size(0)
        # Calculate total sample in data_loader
        total_samples += images.size(0)
        # Calculte y_predict for evaluation
        predicted = torch.argmax(outputs, dim=1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.detach().cpu().numpy())
    loss = running_loss/total_samples
    accuracy = accuracy_score(y_true, y_pred)
    writer.add_scalar("train/loss", loss, epoch)
    writer.add_scalar("train/accuracy", accuracy, epoch)
    return loss, accuracy

# Eval model


In [5]:
def eval_model(model, loss_function, test_dataloader, writer, epoch):
    model.eval()
    total_samples = 0
    total_loss = 0
    y_true = []
    y_pred = []
    with torch.no_grad():
        for images, labels in test_dataloader:
            images = images.to(device)
            labels = labels.to(device)

            # Calculate logits
            outputs = model(images)

            # Calculate loss of outputs and y_true
            loss = loss_function(outputs, labels)
            total_loss += loss.item() * images.size(0)
            # Calculate total sample
            total_samples += images.size(0)

            # Calculte y_predict for evaluation
            predicted = torch.argmax(outputs, dim=1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.detach().cpu().numpy())
    loss = total_loss/total_samples
    accuracy = accuracy_score(y_true, y_pred)
    writer.add_scalar("test/loss", loss, epoch)
    writer.add_scalar("test/accuracy", accuracy, epoch)
    return loss, accuracy

# Model


In [6]:
model = torchvision.models.resnet34(num_classes=2)
model = model.to(device)
print(device)

cuda:0


# Optimizer


In [7]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Loss function


In [8]:
loss_function = torch.nn.CrossEntropyLoss()

# Train


In [9]:
model = torchvision.models.efficientnet_b4(num_classes=2)
model = model.to(device)
model_name = "efficientnet_b4"
list_root_dir = []
list_root_dir.append(os.path.join(os.path.dirname(os.getcwd()), "dataset", "gan_makeup_data_96"))
list_root_dir.append(os.path.join(os.path.dirname(os.getcwd()), "dataset", "mtdataset_96"))
list_root_dir.append(os.path.join(os.path.dirname(
    os.getcwd()), "dataset", "data_anh_Vinh"))

train_transform = torchvision.transforms.Compose([
    torchvision.transforms.RandomResizedCrop([96, 96]),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomVerticalFlip(),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])
test_transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize([96, 96]),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])
writer = SummaryWriter(f"logs/{model_name}")
epoch = 70

# Perform train-test split
train_dataset = ImageDataset(list_root_dir, train_transform)
test_dataset = ImageDataset(list_root_dir, test_transform)

indices = list(range(len(train_dataset)))
train_indices, test_indices = train_test_split(indices, test_size=0.1, train_size=0.9)
train_dataset = torch.utils.data.Subset(train_dataset, train_indices)
test_dataset = torch.utils.data.Subset(test_dataset, test_indices)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

# Keep track best result
lowest_train_loss = math.inf
highest_train_acc = 0
lowest_test_loss = math.inf
highest_test_acc = 0
for i in range(epoch):
    # Train phase
    train_loss, train_acc = train_model(model, loss_function, optimizer, train_loader, writer, i)
    
    # Test phase
    test_loss, test_acc = eval_model(model, loss_function, test_loader, writer, i)
    
    # Command line log
    print(f'''epoch {i}: train_loss {round(train_loss,4)}, train_acc {round(train_acc,4)}, test_loss {test_loss}, test_acc {round(test_acc,4)}''')

    # Save the best model
    if train_loss <= lowest_train_loss and train_acc >= highest_train_acc and test_loss <= lowest_test_loss and test_acc >= highest_test_acc:
        lowest_train_loss = train_loss
        highest_train_acc = train_acc
        lowest_test_loss = test_loss
        highest_test_acc = test_acc
        saved_folder = os.path.join(os.getcwd(),f"{model_name}")
        if not os.path.exists(saved_folder):
            os.makedirs(saved_folder)
        saved_path = os.path.join(saved_folder, f"{i}.pth")
        torch.save(model.state_dict(), saved_path)


writer.flush()
writer.close()

epoch 0: train_loss 2.777, train_acc 0.4872, test_loss 21.2215432580254, test_acc 0.4753
epoch 1: train_loss 2.8642, train_acc 0.4838, test_loss 13.375316521585315, test_acc 0.4945
epoch 2: train_loss 2.7841, train_acc 0.4865, test_loss 12.76891793967165, test_acc 0.4753
epoch 3: train_loss 2.7789, train_acc 0.482, test_loss 10.485060720600638, test_acc 0.5
epoch 4: train_loss 2.7337, train_acc 0.4853, test_loss 11.041288324531298, test_acc 0.4954


In [ ]:
model = torchvision.models.efficientnet_v2_s(num_classes=2)
model = model.to(device)
model_name = "efficientnet_v2_s"
list_root_dir = []
list_root_dir.append(os.path.join(os.path.dirname(os.getcwd()), "dataset", "gan_makeup_data_96"))
list_root_dir.append(os.path.join(os.path.dirname(os.getcwd()), "dataset", "mtdataset_96"))
list_root_dir.append(os.path.join(os.path.dirname(
    os.getcwd()), "dataset", "data_anh_Vinh"))

train_transform = torchvision.transforms.Compose([
    torchvision.transforms.RandomResizedCrop([96, 96]),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomVerticalFlip(),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])
test_transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize([96, 96]),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])
writer = SummaryWriter(f"logs/{model_name}")
epoch = 70

# Perform train-test split
train_dataset = ImageDataset(list_root_dir, train_transform)
test_dataset = ImageDataset(list_root_dir, test_transform)

indices = list(range(len(train_dataset)))
train_indices, test_indices = train_test_split(indices, test_size=0.1, train_size=0.9)
train_dataset = torch.utils.data.Subset(train_dataset, train_indices)
test_dataset = torch.utils.data.Subset(test_dataset, test_indices)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

# Keep track best result
lowest_train_loss = math.inf
highest_train_acc = 0
lowest_test_loss = math.inf
highest_test_acc = 0
for i in range(epoch):
    # Train phase
    train_loss, train_acc = train_model(model, loss_function, optimizer, train_loader, writer, i)
    
    # Test phase
    test_loss, test_acc = eval_model(model, loss_function, test_loader, writer, i)
    
    # Command line log
    print(f'''epoch {i}: train_loss {round(train_loss,4)}, train_acc {round(train_acc,4)}, test_loss {test_loss}, test_acc {round(test_acc,4)}''')

    # Save the best model
    if train_loss <= lowest_train_loss and train_acc >= highest_train_acc and test_loss <= lowest_test_loss and test_acc >= highest_test_acc:
        lowest_train_loss = train_loss
        highest_train_acc = train_acc
        lowest_test_loss = test_loss
        highest_test_acc = test_acc
        saved_folder = os.path.join(os.getcwd(),f"{model_name}")
        if not os.path.exists(saved_folder):
            os.makedirs(saved_folder)
        saved_path = os.path.join(saved_folder, f"{i}.pth")
        torch.save(model.state_dict(), saved_path)


writer.flush()
writer.close()

KeyboardInterrupt: 

In [ ]:
model = torchvision.models.efficientnet_v2_m(num_classes=2)
model = model.to(device)
model_name = "efficientnet_v2_m"
list_root_dir = []
list_root_dir.append(os.path.join(os.path.dirname(os.getcwd()), "dataset", "gan_makeup_data_96"))
list_root_dir.append(os.path.join(os.path.dirname(os.getcwd()), "dataset", "mtdataset_96"))
list_root_dir.append(os.path.join(os.path.dirname(
    os.getcwd()), "dataset", "data_anh_Vinh"))

train_transform = torchvision.transforms.Compose([
    torchvision.transforms.RandomResizedCrop([96, 96]),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomVerticalFlip(),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])
test_transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize([96, 96]),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])
writer = SummaryWriter(f"logs/{model_name}")
epoch = 70

# Perform train-test split
train_dataset = ImageDataset(list_root_dir, train_transform)
test_dataset = ImageDataset(list_root_dir, test_transform)

indices = list(range(len(train_dataset)))
train_indices, test_indices = train_test_split(indices, test_size=0.1, train_size=0.9)
train_dataset = torch.utils.data.Subset(train_dataset, train_indices)
test_dataset = torch.utils.data.Subset(test_dataset, test_indices)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

# Keep track best result
lowest_train_loss = math.inf
highest_train_acc = 0
lowest_test_loss = math.inf
highest_test_acc = 0
for i in range(epoch):
    # Train phase
    train_loss, train_acc = train_model(model, loss_function, optimizer, train_loader, writer, i)
    
    # Test phase
    test_loss, test_acc = eval_model(model, loss_function, test_loader, writer, i)
    
    # Command line log
    print(f'''epoch {i}: train_loss {round(train_loss,4)}, train_acc {round(train_acc,4)}, test_loss {test_loss}, test_acc {round(test_acc,4)}''')

    # Save the best model
    if train_loss <= lowest_train_loss and train_acc >= highest_train_acc and test_loss <= lowest_test_loss and test_acc >= highest_test_acc:
        lowest_train_loss = train_loss
        highest_train_acc = train_acc
        lowest_test_loss = test_loss
        highest_test_acc = test_acc
        saved_folder = os.path.join(os.getcwd(),f"{model_name}")
        if not os.path.exists(saved_folder):
            os.makedirs(saved_folder)
        saved_path = os.path.join(saved_folder, f"{i}.pth")
        torch.save(model.state_dict(), saved_path)


writer.flush()
writer.close()

In [ ]:
model = torchvision.models.efficientnet_v2_l(num_classes=2)
model = model.to(device)
model_name = "efficientnet_v2_l"
list_root_dir = []
list_root_dir.append(os.path.join(os.path.dirname(os.getcwd()), "dataset", "gan_makeup_data_96"))
list_root_dir.append(os.path.join(os.path.dirname(os.getcwd()), "dataset", "mtdataset_96"))
list_root_dir.append(os.path.join(os.path.dirname(
    os.getcwd()), "dataset", "data_anh_Vinh"))

train_transform = torchvision.transforms.Compose([
    torchvision.transforms.RandomResizedCrop([96, 96]),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomVerticalFlip(),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])
test_transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize([96, 96]),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])
writer = SummaryWriter(f"logs/{model_name}")
epoch = 70

# Perform train-test split
train_dataset = ImageDataset(list_root_dir, train_transform)
test_dataset = ImageDataset(list_root_dir, test_transform)

indices = list(range(len(train_dataset)))
train_indices, test_indices = train_test_split(indices, test_size=0.1, train_size=0.9)
train_dataset = torch.utils.data.Subset(train_dataset, train_indices)
test_dataset = torch.utils.data.Subset(test_dataset, test_indices)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

# Keep track best result
lowest_train_loss = math.inf
highest_train_acc = 0
lowest_test_loss = math.inf
highest_test_acc = 0
for i in range(epoch):
    # Train phase
    train_loss, train_acc = train_model(model, loss_function, optimizer, train_loader, writer, i)
    
    # Test phase
    test_loss, test_acc = eval_model(model, loss_function, test_loader, writer, i)
    
    # Command line log
    print(f'''epoch {i}: train_loss {round(train_loss,4)}, train_acc {round(train_acc,4)}, test_loss {test_loss}, test_acc {round(test_acc,4)}''')

    # Save the best model
    if train_loss <= lowest_train_loss and train_acc >= highest_train_acc and test_loss <= lowest_test_loss and test_acc >= highest_test_acc:
        lowest_train_loss = train_loss
        highest_train_acc = train_acc
        lowest_test_loss = test_loss
        highest_test_acc = test_acc
        saved_folder = os.path.join(os.getcwd(),f"{model_name}")
        if not os.path.exists(saved_folder):
            os.makedirs(saved_folder)
        saved_path = os.path.join(saved_folder, f"{i}.pth")
        torch.save(model.state_dict(), saved_path)


writer.flush()
writer.close()